# Westeros Tutorial - Introducing emissions (part1/2): Adding emission bounds

In the first part, the model chose not base the power system on wind power as electricity from  wind turbines was more expensive than electricity produced from coal. However, now we introduce emissions to investigate the impact of climate policy. Let's see what happens then.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform()

In [ ]:
model = 'Westeros Electrified'

base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'emission_pool_test','introducing an upper bound on emissions',
                  keep_solution=False)
scen.check_out()

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Introducing Emissions

In [ ]:
# first we introduce the emission of CO2 and the emission category GHG
scen.add_set('emission', 'CO2')
scen.add_cat('emission', 'GHG', 'CO2')

# we now add CO2 emissions to the coal powerplant
base_emission_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'unit': 'tCO2/kWa',
}

# adding new units to the model library (needed only once)
mp.add_unit('tCO2/kWa')
mp.add_unit('MtCO2')

emission_factor = make_df(base_emission_factor, technology= 'coal_ppl', emission= 'CO2', value = 7.4)
scen.add_par('emission_factor', emission_factor)

## Define a Bound on Emissions

The `type_year: cumulative` assigns an upper bound on the *weighted average of emissions* over the entire time horizon.

In [ ]:
scen.add_par('bound_emission', [country, 'GHG', 'all', 'cumulative'],
             value=500., unit='MtCO2')

## Initialize parameters

In [ ]:
new_var = {'EMISS_POOL': ['node', 'type_emission', 'type_tec', 'year']}
for var, idx in new_var.items():
    if var not in scen.var_list():
        scen.init_var(var, idx)

## Add GHG poolsize for historical period 690

In [ ]:
scen.add_set('type_year', 690)
scen.add_set('cat_year', pd.DataFrame({'type_year': [690], 'year': 690}))
df = pd.DataFrame({
    'node': 'Westeros',
    'type_emission': 'GHG',
    'type_tec': 'all',
    'year': 690,
    'value': [100000],
    'unit': 'MtCO2'
})
scen.add_par('historical_emission_pool', df)

## Add sinkrate for model-periods

In [ ]:
df = pd.DataFrame({
    'node': 'Westeros',
    'type_emission': 'GHG',
    'type_tec': 'all',
    'year': [700, 710, 720],
    'value': [0.02, 0.01, 0.005],
    'unit': '???'
})
scen.add_par('emission_sink_rate', df)

## Add is_sinkrate for model-periods

In [ ]:
# Adding mapping sets of new parameters
if not df.empty:
    for i in df.index:
        d = df.loc[i, :].copy().drop(['value', 'unit'])
        scen.add_set('is_emission_sink', d)

## OPTIONAL: Add `tax_emission_pool`

In [ ]:
add_tep = False
if add_tep:
    df = pd.DataFrame({
        'node': 'Westeros',
        'type_emission': 'GHG',
        'type_tec': 'all',
        'year': [700, 710, 720],
        'value': [500, 500, 500],
        'unit': '???'
    })
    scen.add_par('tax_emission_pool', df) 

## OPTIONAL: Add `bound_emission_pool`

In [ ]:
add_epb = False
if add_epb:
    df = pd.DataFrame({
        'node': 'Westeros',
        'type_emission': 'GHG',
        'type_tec': 'all',
        'year': [700, 710, 720],
        'value': [87000, 86000, 85000],
        'unit': '???'
    })
    scen.add_par('bound_emission_pool', df)

    # Adding mapping sets of new parameters
    if not df.empty:
        for i in df.index:
            d = df.loc[i, :].copy().drop(['value', 'unit'])
            scen.add_set('is_bound_emission_pool', d)
            
    # Remove bound_emission
    df = scen.par('bound_emission')
    scen.remove_par('bound_emission', df)

## Time to Solve the Model

In [ ]:
scen.commit(comment='introducing emissions and setting an upper bound')
scen.set_as_default()

In [ ]:
scen.solve(var_list=['EMISS_POOL'])

In [ ]:
scen.var('OBJ')['lvl']

## Checking Results

In [ ]:
#    WITHOUT any constraints EMISS_POOL should have results:
#    700 88299.738226
#    710 86699.717263
#    720 84447.695837 
    
#    WITH `tax_emission_pool' EMISS_POOL should have results:
#    700 86874.248590
#    710 80820.292479
#    720 76971.707123

#    WITH `bound_emission_pool' EMISS_POOL should have results:
#    700 87000
#    710 85975.299326
#    720 85000

In [ ]:
scen.var('EMISS_POOL')

## Close the connection to the database

In [ ]:
mp.close_db()